# Basic dense (fully connected) neural network models



### Basic simple neural network model

In the previous notebooks, we saw how to implement simple neural network models with **just the output layer** for logistic, softmax (multiclass) and linear regression problems: this output layer had **only one node** (logistic and linear regression) which performed both the linear combination of input variables + bias and the sigmoid/linear activation:

<img src="https://drive.google.com/uc?export=view&id=1PRc719uT1kOUuCMbpHML2sEk7qp6UJnm">

(Softmax regression is slightly different: the single output layer has as many nodes as there are classes, each calculating the linear combination of input variables and the softmax activation).


### Basic dense neural network model

We are now building a **neural network model**, by adding **one hidden layer** (not deep) with **u nodes** (units):

<img src="https://drive.google.com/uc?export=view&id=1QROz9pFnMoqTeqrFbele8pFz8qXDSckq">

There's a number of `hyperparameters`:

- the **number of hidden nodes** (number of units in the hidden layer)
- the **type of activation function** in the hidden layer
- the **output activation function**
- the **loss function** (for backpropagation)
- the **optimizer** (for gradient descent)

By stacking together more than one hidden/intermediate layer (additional hyperparameter), we can then build **deep neural networks**.

## Loading libraries and setting the random seed

First of all, we load some necessary libraries; then we setup the random seed to ensure reproducibility of results. Since tensorflow uses an internal random generator we need to fix both the general seed (via numpy `seed()`) and tensorflow seed (via `set_seet()`)

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

  # Set the seed using keras.utils.set_random_seed. This will set:
  # 1) `numpy` seed
  # 2) `tensorflow` random seed
  # 3) `python` random seed
tf.keras.utils.set_random_seed(10)

  # This will make TensorFlow ops as deterministic as possible, but it will
  # affect the overall performance, so it's not enabled by default.
  # `enable_op_determinism()` is introduced in TensorFlow 2.9.
tf.config.experimental.enable_op_determinism()

## Get the data

Example data

In [3]:
import sklearn.datasets

(features, target) = sklearn.datasets.load_iris(return_X_y = True) ## feature names are not returned
print(features.shape)
print(target.shape)

(150, 4)
(150,)


In [ ]:
## # Configuration options
input_shape = (features.shape[1],) ## tuple that specifies the number of features
hidden_nodes = 8
hidden_activation = 'relu'
output_activation = 'sigmoid'
loss_function = 'binary_crossentropy'
optimizer_used = 'SGD' ##stochastic gradient descent
num_epochs = 200

[TODO]